# Handwritten Digit Classfication using Deep Feed Forward Neural Network

In this example, we are going to use the MNIST dataset to train a multi-layer feed foward neural network. MNIST is a simple computer vision dataset of handwritten digits. It has 60,000 training examles and 10,000 test examples. "It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting." For more details, please checkout the website [MNIST](http://yann.lecun.com/exdb/mnist/)

In [1]:
import org.apache.log4j.{Level, Logger}
import org.apache.spark.SparkContext

import com.intel.analytics.bigdl.utils._
import com.intel.analytics.bigdl.utils.{Engine, LoggerFilter, T, Table}
import com.intel.analytics.bigdl.dataset.DataSet
import com.intel.analytics.bigdl.dataset.image.{BytesToGreyImg, GreyImgNormalizer, GreyImgToBatch, GreyImgToSample}
import com.intel.analytics.bigdl.models.lenet.Utils._
import com.intel.analytics.bigdl.nn._
import com.intel.analytics.bigdl.optim._
import com.intel.analytics.bigdl.optim.{Adam, Top1Accuracy, Trigger}
import com.intel.analytics.bigdl.visualization.{TrainSummary, ValidationSummary}
import com.intel.analytics.bigdl.tensor.Tensor
import com.intel.analytics.bigdl.numeric.NumericFloat

Engine.init

## 1. Load MNIST dataset

In [2]:
import java.nio.ByteBuffer
import java.nio.file.{Files, Path, Paths}

import com.intel.analytics.bigdl.dataset.ByteRecord
import com.intel.analytics.bigdl.utils.File
import scopt.OptionParser

def load(featureFile: String, labelFile: String): Array[ByteRecord] = {
    val featureBuffer = ByteBuffer.wrap(Files.readAllBytes(Paths.get(featureFile)))
    val labelBuffer = ByteBuffer.wrap(Files.readAllBytes(Paths.get(labelFile)))
    
    val labelMagicNumber = labelBuffer.getInt()
    require(labelMagicNumber == 2049)
    val featureMagicNumber = featureBuffer.getInt()
    require(featureMagicNumber == 2051)

    val labelCount = labelBuffer.getInt()
    val featureCount = featureBuffer.getInt()
    require(labelCount == featureCount)

    val rowNum = featureBuffer.getInt()
    val colNum = featureBuffer.getInt()

    val result = new Array[ByteRecord](featureCount)
    var i = 0
    while (i < featureCount) {
      val img = new Array[Byte]((rowNum * colNum))
      var y = 0
      while (y < rowNum) {
        var x = 0
        while (x < colNum) {
          img(x + y * colNum) = featureBuffer.get()
          x += 1
        }
        y += 1
      }
      result(i) = ByteRecord(img, labelBuffer.get().toFloat + 1.0f)
      i += 1
    }

    result
}

Then we need to set paths of data. Please edit paths if they are changed.

In [3]:
val trainData = "../../datasets/mnist/train-images-idx3-ubyte"
val trainLabel = "../../datasets/mnist/train-labels-idx1-ubyte"
val validationData = "../../datasets/mnist/t10k-images-idx3-ubyte"
val validationLabel = "../../datasets/mnist/t10k-labels-idx1-ubyte"

## 2. Deep Feed Forward Neural Network Model Setup

This time we will use the deep feed forward neural network to classify handwritten digits. You can checkout this blog to get a detailed understanding of the deep feed forward neural network in particular.

In [4]:
//Parameters
val learningRate = 0.2
val batchSize = 2048
val maxEpochs = 15
val displayStep = 1

//Network Parameters
val nInput = 784 //MNIST data input (img shape: 28*28)
val nHidden1 = 256 // 1st hidden layer num of features
val nHidden2 = 256 // 2nd hidden layer num of features
val nClasses = 10  //MNIST total classes (0-9 digits)

Then the data set should be created and the model needs to be established.

In [5]:
val trainSet = 
    DataSet.array(load(trainData, trainLabel), sc) -> BytesToGreyImg(28, 28) -> GreyImgNormalizer(trainMean, trainStd) -> GreyImgToBatch(batchSize)
val validationSet = 
    DataSet.array(load(validationData, validationLabel), sc) -> BytesToGreyImg(28, 28) -> GreyImgNormalizer(testMean, testStd) -> GreyImgToBatch(batchSize)

In [6]:
val model = Sequential()
model.add(Reshape(Array(28*28)))
model.add(Linear(nInput, nHidden1).setName("mlp_fc1"))
model.add(ReLU())
// Hidden layer with ReLu activation
model.add(Linear(nHidden1, nHidden2).setName("mlp_fc2"))
model.add(ReLU())
// output layer
model.add(Linear(nHidden2, nClasses).setName("mlp_fc3"))
model.add(LogSoftMax())

Sequential[6bd56665]{
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> output]
  (1): Reshape[db28b021](784)
  (2): Linear[mlp_fc1](784 -> 256)
  (3): ReLU[8cf55aa1](0.0, 0.0)
  (4): Linear[mlp_fc2](256 -> 256)
  (5): ReLU[53e45a6d](0.0, 0.0)
  (6): Linear[mlp_fc3](256 -> 10)
  (7): LogSoftMax[d5d9df87]
}

## 3. Optimizer Setup

In [7]:
val optimizer = Optimizer(model = model, dataset = trainSet, criterion = ClassNLLCriterion[Float]())
optimizer.setValidation(trigger = Trigger.everyEpoch, dataset = validationSet, vMethods = Array(new Top1Accuracy))
optimizer.setOptimMethod(new SGD(learningRate=learningRate))
optimizer.setEndWhen(Trigger.maxEpoch(maxEpochs))

com.intel.analytics.bigdl.optim.DistriOptimizer@466152cb

The following is to create training and validation summary.

In [8]:
import java.text.SimpleDateFormat
import java.util.Calendar
val today = Calendar.getInstance
val formatDate = new SimpleDateFormat("yyyyMMdd-hhmmss")
val name = "multilayer_perceptron-" + formatDate.format(today.getTime()).toString()
val trainSummary = TrainSummary(logDir="/tmp/bigdl_summaries", appName=name)
trainSummary.setSummaryTrigger("Parameters", Trigger.severalIteration(50))
val valSummary = ValidationSummary(logDir="/tmp/bigdl_summaries", appName=name)
optimizer.setTrainSummary(trainSummary)
optimizer.setValidationSummary(valSummary)
printf("saving logs to %s", name)

saving logs to multilayer_perceptron-20170922-012115

In [9]:
// Boot training process
val trainedModel = optimizer.optimize()
print("Optimization Done.")

can't find locality partition for partition 0 Partition locations are (ArrayBuffer(172.168.2.109)) Candidate partition locations are
(0,List()).
Optimization Done.

In [10]:
val rddData = sc.parallelize(load(validationData, validationLabel), batchSize)
val transformer = BytesToGreyImg(28, 28) -> GreyImgNormalizer(testMean, testStd) -> GreyImgToSample()
val evaluationSet = transformer(rddData)
        
val result = trainedModel.evaluate(evaluationSet, Array(new Top1Accuracy[Float]), Some(batchSize))

result.foreach(r => println(s"${r._2} is ${r._1}"))

Top1Accuracy is Accuracy(correct: 9670, count: 10000, accuracy: 0.967)


In [11]:
val predictions = trainedModel.predict(evaluationSet)
val preLables = predictions.take(8).map(_.toTensor.max(1)._2.valueAt(1)).mkString(",")
val lables = evaluationSet.take(8).map(_.label.valueAt(1)).mkString(",")
println(preLables)
println(lables)

8.0,3.0,2.0,1.0,5.0,2.0,5.0,10.0
8.0,3.0,2.0,1.0,5.0,2.0,5.0,10.0


## 4. Draw the performance curve

In [12]:
import vegas._
import vegas.render.HTMLRenderer._
import vegas.DSL._

In [13]:
val loss = trainSummary.readScalar("Loss")
val top1 = valSummary.readScalar("Top1Accuracy")

val lossXY = loss.map(_ ._1).zip(loss.map(_ ._2)).toSeq

Vegas(description = "The Loss curve.", width = 700.0, height = 300.0).
  withXY(lossXY).
  encodeX("x", Quantitative, bin = Bin(maxbins = 500.0), title = "Iteration").
  encodeY("y", Quantitative, title = "Loss").
  mark(Line).
  show

val top1XY = top1.map(_ ._1).zip(top1.map(_ ._2)).toSeq
Vegas(description = "The Top1Accuracy curve.", width = 700.0, height = 300.0).
  withXY(top1XY).
  encodeX("x", Quantitative, bin = Bin(maxbins = 500.0), title = "Iteration").
  encodeY("y", Quantitative, bin = Bin(base = 0.9), title = "Top1Accuracy").
  mark(Line).
  show

<iframe id="frame-vegas-c94a3463-f0c0-4ea6-91cf-072360300189" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-c94a3463-f0c0-4ea6-91cf-072360300189'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "width" : 700.0,
 "height" : 300.0,
 "mark" : "line",
 "encoding" : {
 "x" : {
 "field" : "x",
 "type" : "quantitative",
 "bin" : {
 "maxbins" : 500.0
 },
 "title" : "Iteration"
 },
 "y" : {
 "field" : "y",
 "type" : "quantitative",
 "title" : "Loss"
 }
 },
 "description" : "The Loss curve.",
 "data" : {
 "values" : [
 {
 "x" : 1,
 "y" : 2.3121957778930664
 },
 {
 "x" : 2,
 "y" : 2.2584104537963867
 },
 {
 "x" : 3,
 "y" : 2.200562000274658
 },
 {
 "x" : 4,
 "y" : 2.136345148086548
 },
 {
 "x" : 5,
 "y" : 2.0669374465942383
 },
 {
 "x" : 6,
 "y" : 1.9538155794143677
 },
 {
 "x" : 7,
 "y" : 1.8442598581314087
 },
 {
 "x" : 8,
 "y" : 1.6962454319000244
 },
 {
 "x" : 9,
 "y" : 1.506381630897522
 },
 {
 "x" : 10,
 "y" : 1.3512290716171265
 },
 {
 "x" : 11,
 "y" : 1.1823108196258545
 },
 {
 "x" : 12,
 "y" : 1.113419771194458
 },
 {
 "x" : 13,
 "y" : 1.0526624917984009
 },
 {
 "x" : 14,
 "y" : 1.0102297067642212
 },
 {
 "x" : 15,
 "y" : 1.1379748582839966
 },
 {
 "x" : 16,
 "y" : 1.1985751390457153
 },
 {
 "x" : 17,
 "y" : 1.1047813892364502
 },
 {
 "x" : 18,
 "y" : 0.9947450757026672
 },
 {
 "x" : 19,
 "y" : 0.8474062085151672
 },
 {
 "x" : 20,
 "y" : 0.7556551098823547
 },
 {
 "x" : 21,
 "y" : 0.7605059146881104
 },
 {
 "x" : 22,
 "y" : 0.7993674278259277
 },
 {
 "x" : 23,
 "y" : 0.831486701965332
 },
 {
 "x" : 24,
 "y" : 0.7468907833099365
 },
 {
 "x" : 25,
 "y" : 0.6180701851844788
 },
 {
 "x" : 26,
 "y" : 0.5729503035545349
 },
 {
 "x" : 27,
 "y" : 0.5147413015365601
 },
 {
 "x" : 28,
 "y" : 0.5408098697662354
 },
 {
 "x" : 29,
 "y" : 0.510824978351593
 },
 {
 "x" : 30,
 "y" : 0.5335638523101807
 },
 {
 "x" : 31,
 "y" : 0.5174471139907837
 },
 {
 "x" : 32,
 "y" : 0.5219124555587769
 },
 {
 "x" : 33,
 "y" : 0.5723028182983398
 },
 {
 "x" : 34,
 "y" : 0.6022515296936035
 },
 {
 "x" : 35,
 "y" : 0.6064257621765137
 },
 {
 "x" : 36,
 "y" : 0.5266075730323792
 },
 {
 "x" : 37,
 "y" : 0.4591144919395447
 },
 {
 "x" : 38,
 "y" : 0.479391872882843
 },
 {
 "x" : 39,
 "y" : 0.4734329879283905
 },
 {
 "x" : 40,
 "y" : 0.6228542327880859
 },
 {
 "x" : 41,
 "y" : 0.6711607575416565
 },
 {
 "x" : 42,
 "y" : 0.5061947703361511
 },
 {
 "x" : 43,
 "y" : 0.4694098234176636
 },
 {
 "x" : 44,
 "y" : 0.37212038040161133
 },
 {
 "x" : 45,
 "y" : 0.37654078006744385
 },
 {
 "x" : 46,
 "y" : 0.34541794657707214
 },
 {
 "x" : 47,
 "y" : 0.34755292534828186
 },
 {
 "x" : 48,
 "y" : 0.3294471800327301
 },
 {
 "x" : 49,
 "y" : 0.3253633379936218
 },
 {
 "x" : 50,
 "y" : 0.3561883866786957
 },
 {
 "x" : 51,
 "y" : 0.32957005500793457
 },
 {
 "x" : 52,
 "y" : 0.33822929859161377
 },
 {
 "x" : 53,
 "y" : 0.34359845519065857
 },
 {
 "x" : 54,
 "y" : 0.30979350209236145
 },
 {
 "x" : 55,
 "y" : 0.35926973819732666
 },
 {
 "x" : 56,
 "y" : 0.31505927443504333
 },
 {
 "x" : 57,
 "y" : 0.30715134739875793
 },
 {
 "x" : 58,
 "y" : 0.36267465353012085
 },
 {
 "x" : 59,
 "y" : 0.33162105083465576
 },
 {
 "x" : 60,
 "y" : 0.3377307653427124
 },
 {
 "x" : 61,
 "y" : 0.32808974385261536
 },
 {
 "x" : 62,
 "y" : 0.3604009747505188
 },
 {
 "x" : 63,
 "y" : 0.3815775513648987
 },
 {
 "x" : 64,
 "y" : 0.34143123030662537
 },
 {
 "x" : 65,
 "y" : 0.3233301043510437
 },
 {
 "x" : 66,
 "y" : 0.37334853410720825
 },
 {
 "x" : 67,
 "y" : 0.379881680

<iframe id="frame-vegas-1376a79c-8218-4d6b-8741-04ad3fa8d099" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-1376a79c-8218-4d6b-8741-04ad3fa8d099'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "width" : 700.0,
 "height" : 300.0,
 "mark" : "line",
 "encoding" : {
 "x" : {
 "field" : "x",
 "type" : "quantitative",
 "bin" : {
 "maxbins" : 500.0
 },
 "title" : "Iteration"
 },
 "y" : {
 "field" : "y",
 "type" : "quantitative",
 "bin" : {
 "base" : 0.9
 },
 "title" : "Top1Accuracy"
 }
 },
 "description" : "The Top1Accuracy curve.",
 "data" : {
 "values" : [
 {
 "x" : 30,
 "y" : 0.8414000272750854
 },
 {
 "x" : 60,
 "y" : 0.9018999934196472
 },
 {
 "x" : 90,
 "y" : 0.9246000051498413
 },
 {
 "x" : 120,
 "y" : 0.929099977016449
 },
 {
 "x" : 150,
 "y" : 0.9377999901771545
 },
 {
 "x" : 180,
 "y" : 0.9429000020027161
 },
 {
 "x" : 210,
 "y" : 0.9476000070571899
 },
 {
 "x" : 240,
 "y" : 0.9556000232696533
 },
 {
 "x" : 270,
 "y" : 0.9531999826431274
 },
 {
 "x" : 300,
 "y" : 0.9585999846458435
 },
 {
 "x" : 330,
 "y" : 0.9599000215530396
 },
 {
 "x" : 360,
 "y" : 0.9625999927520752
 },
 {
 "x" : 390,
 "y" : 0.9632999897003174
 },
 {
 "x" : 420,
 "y" : 0.965499997138977
 },
 {
 "x" : 450,
 "y" : 0.9672999978065491
 }
 ]
 }
}
 }
 vg.embed("#vegas-1376a79c-8218-4d6b-8741-04ad3fa8d099", embedSpec, function(error, result) {});
 </script>

 </body>
</html>">

Finally, the Spark should be stopped.

In [14]:
sc.stop()